In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
from tensorflow.python.client import device_lib
from collections import Counter
from math import inf

In [2]:
f = open('../../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()
word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

In [3]:
f = open("../../../dataset/sense/dict_sense-keys", 'rb')
dict_sense_keys = pickle.load(f)
f.close()

f = open("../../../dataset/sense/dict_word-sense", 'rb')
dict_word_sense = pickle.load(f)
f.close()

In [4]:
# Model Description
sense_word = 'place'
model_name = 'model-4'
sense_word_dir = '../output/' + sense_word
model_dir = sense_word_dir + '/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(sense_word_dir):
    os.mkdir(sense_word_dir)
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

In [5]:
f = open("../../../dataset/checkwords/"+ sense_word + "_data", 'rb')
data = pickle.load(f)
f.close()

data_y = []
for i in range(len(data)):
    data_y.append(dict_sense_keys[data[i][0]][3])

sense_count = Counter(data_y)
sense_count = sense_count.most_common()[:6]
vocab_sense = [k for k,v in sense_count]
vocab_sense = sorted(vocab_sense, key=lambda x:int(x[0]))
sense_count,vocab_sense

([('1:15', 1149),
  ('2:35', 623),
  ('2:41', 490),
  ('1:26', 488),
  ('2:31', 479),
  ('1:04', 282)],
 ['1:15', '1:26', '1:04', '2:35', '2:41', '2:31'])

In [6]:
def make_mask_matrix(sense_word,vocab_sense):
    mask_mat = []
    sense_list = [int(string[0]) for string in vocab_sense]
    sense_count = list(Counter(sense_list).values())
    start=0
    prev=0
    for i in range(len(set(sense_list))):
        temp_row=[0]*len(sense_list)
        for j in range(len(sense_list)):
            if j>=start and j<sense_count[i]+prev:
                temp_row[j]= 0
            else:
                temp_row[j]= -10
        start+=sense_count[i]
        prev+=sense_count[i]
        mask_mat.append(temp_row)
    return mask_mat

In [7]:
vocab_sense

['1:15', '1:26', '1:04', '2:35', '2:41', '2:31']

In [8]:
mask_mat = make_mask_matrix(sense_word,vocab_sense)
mask_mat

[[0, 0, 0, -10, -10, -10], [-10, -10, -10, 0, 0, 0]]

In [9]:
data_x = []
data_label = []
data_pos = []
for i in range(len(data)):
    if dict_sense_keys[data[i][0]][3] in vocab_sense:
        data_x.append(data[i][1])
        data_label.append(dict_sense_keys[data[i][0]][3])
        data_pos.append(dict_sense_keys[data[i][0]][1])


print(len(data_label), len(data_y))

# vocab_sense = dict_word_sense[sense_word]

sense2id = dict((s, i) for i,s in enumerate(vocab_sense))
id2sense = dict((i, s) for i,s in enumerate(vocab_sense))

count_pos = Counter(data_pos)
count_pos = count_pos.most_common()
vocab_pos = [int(k) for k,v in count_pos]
vocab_pos = sorted(vocab_pos, key=lambda x:int(x))

pos2id = dict((str(s), i) for i,s in enumerate(vocab_pos))
id2pos = dict((i, str(s)) for i,s in enumerate(vocab_pos))

print(vocab_pos)

3511 3569
[1, 2]


In [10]:
# Parameters
mode = 'train'
num_senses = len(vocab_sense)
num_pos = len(vocab_pos)
batch_size = 64
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 100
keep_prob = 0.5
l2_lambda = 0.001
init_lr = 0.005
decay_steps = 500
decay_rate = 0.96
clip_norm = 1
clipping = True
lambda_loss_pos = 5

In [11]:
# MODEL
def attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
    return c

x = tf.placeholder('int32', [batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [batch_size], name="y")
y_pos = tf.placeholder('int32', [batch_size], name="y_pos")
x_mask  = tf.placeholder('bool', [batch_size, max_sent_size], name='x_mask') 
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 1)

mask_matrix = tf.constant(value=mask_mat, shape=list(np.array(mask_mat).shape), dtype='float32') # mask_matrix

with tf.name_scope("word_embedding"):
    if mode == 'train':
        unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
    else:
        unk_word_emb_mat = tf.get_variable("word_emb_mat", shape=[unk_vocab_size, word_emb_size], dtype='float')
        
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)
    Wx = tf.nn.embedding_lookup(final_word_emb_mat, x)  

with tf.variable_scope("lstm1"):
    cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
    d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)
    
    (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
    h1 = tf.concat([fw_h1, bw_h1], 2)
    
with tf.variable_scope("lstm2"):
    cell_fw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw2 = tf.contrib.rnn.DropoutWrapper(cell_fw2, input_keep_prob=input_keep_prob)
    d_cell_bw2 = tf.contrib.rnn.DropoutWrapper(cell_bw2, input_keep_prob=input_keep_prob)
    
    (fw_h2, bw_h2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw2, d_cell_bw2, h1, sequence_length=x_len, dtype='float', scope='lstm2')
    h = tf.concat([fw_h2, bw_h2], 2)
    
with tf.variable_scope("attention_pos"):
    W_att1 = tf.Variable(tf.truncated_normal([2*hidden_size, 1], mean=0.0, stddev=0.1, seed=0), name="W_att1")
    c1 = tf.expand_dims(attention(h1[0], x_mask[0], W_att1), 0)
    for i in range(1, batch_size):
        c1 = tf.concat([c1, tf.expand_dims(attention(h1[i], x_mask[i], W_att1), 0)], 0)
        
with tf.variable_scope("softmax_layer_pos"):
    W1 = tf.Variable(tf.truncated_normal([2*hidden_size, num_pos], mean=0.0, stddev=0.1, seed=0), name="W1")
    b1 = tf.Variable(tf.zeros([num_pos]), name="b1")
    drop_c1 = tf.nn.dropout(c1, input_keep_prob)
    logits_pos = tf.matmul(drop_c1, W1) + b1
    predictions_pos = tf.argmax(logits_pos, 1)
    final_masking = tf.nn.embedding_lookup(mask_matrix, predictions_pos)

with tf.variable_scope("attention"):
    W_att = tf.Variable(tf.truncated_normal([2*hidden_size, 1], mean=0.0, stddev=0.1, seed=0), name="W_att")
    c = tf.expand_dims(attention(h[0], x_mask[0], W_att), 0)
    for i in range(1, batch_size):
        c = tf.concat([c, tf.expand_dims(attention(h[i], x_mask[i], W_att), 0)], 0)
        
with tf.variable_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([2*hidden_size, num_senses], mean=0.0, stddev=0.1, seed=0), name="W")
    b = tf.Variable(tf.zeros([num_senses]), name="b")
    drop_c = tf.nn.dropout(c, input_keep_prob)
    logits = tf.matmul(drop_c, W) + b
    masked_logits = logits + final_masking
    predictions = tf.argmax(masked_logits, 1)
    
loss_pos = lambda_loss_pos * tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_pos, labels=y_pos))
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=masked_logits, labels=y))

global_step = tf.Variable(0, trainable=False, name="global_step")

learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

tv_all = tf.trainable_variables()
tv_regu =[]
for t in tv_all:
    if t.name.find('b:')==-1:
        tv_regu.append(t)
        
# l2 Loss
l2_loss = l2_lambda * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])

total_loss = loss + l2_loss + loss_pos

#pos optimizer
pos_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_pos, global_step)

# Optimizer for loss
optimizer = tf.train.AdamOptimizer(learning_rate)

# Gradients and Variables for Loss
grads_vars = optimizer.compute_gradients(total_loss)

# Clipping of Gradients
clipped_grads = grads_vars
if(clipping == True):
    clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

# Training Optimizer for Total Loss
train_op = optimizer.apply_gradients(clipped_grads, global_step=global_step)

# Summaries
var_summaries = []
for v in tv_all:
    var_summary = tf.summary.histogram("{}/var".format(v.name), v)
    var_summaries.append(var_summary)

var_summaries_merged = tf.summary.merge(var_summaries)

loss_summary = tf.summary.scalar("loss", loss)
loss_poss_summary = tf.summary.scalar('loss_pos',loss_pos)
total_loss_summary = tf.summary.scalar("total_loss", total_loss)
summary = tf.summary.merge_all()

In [12]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
saver = tf.train.Saver()                                             # For Saving the model
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [13]:
index = []
for i in range(len(data_x)):
    index.append(i)

In [14]:
index_train, index_val, label_train, label_val = train_test_split(index, data_label, train_size=0.8, shuffle=True, stratify=data_label, random_state=0)

data_x = np.array(data_x)
data_pos = np.array(data_pos)

x_train = data_x[index_train]
pos_train = data_pos[index_train]
x_val = data_x[index_val]
pos_val = data_pos[index_val]

/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [15]:
def data_prepare(x, y, p):
    num_examples = len(x)

    xx = np.zeros([num_examples, max_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_sent_size], dtype=bool)
    yy = np.zeros([num_examples], dtype=int)
    pp = np.zeros([num_examples], dtype=int)
    
    for j in range(num_examples):
        for i in range(max_sent_size):
            if(i>=len(x[j])):
                break
            w = x[j][i]
            xx[j][i] = word2id[w] if w in word2id else word2id['UNKNOWN_TOKEN']
            xx_mask[j][i] = True
        yy[j] = sense2id[y[j]]
        pp[j] = pos2id[p[j]]
    return xx, xx_mask, yy, pp

def eval_score(yy, pred, pp, pred_pos):
    num_batches = int(len(yy)/batch_size)
    f1 = f1_score(yy[:batch_size*num_batches], pred, average='macro')
    accu = accuracy_score(yy[:batch_size*num_batches], pred)
    
    f1_pos = f1_score(pp[:batch_size*num_batches], pred_pos, average='macro')
    accu_pos = accuracy_score(pp[:batch_size*num_batches], pred_pos)
    return f1*100, accu*100, f1_pos*100, accu_pos*100

def model(xx, yy, mask, pp, train_cond=True, pre_train=False):
    num_batches = int(len(xx)/batch_size)
    losses = 0
    preds = []
    pos_preds = []
    for j in range(num_batches): 
        
        s = j * batch_size
        e = (j+1) * batch_size
        
        feed_dict = {x:xx[s:e], y:yy[s:e], y_pos:pp[s:e], x_mask:mask[s:e], is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        if(train_cond==True):
            
            if(pre_train==False):
                _, _loss, step, _summary = sess.run([train_op, total_loss, global_step, summary], feed_dict)
                
            else:
                _, _loss, step, _summary = sess.run([pos_optimizer, loss_pos, global_step, summary], feed_dict)
            
            summary_writer.add_summary(_summary, step)
#             if step%5==0:
#                 print("Steps:{}".format(step), ", Loss: {}".format(_loss))

        else:
            _loss, pred, pred_pos = sess.run([total_loss, predictions, predictions_pos], feed_dict)
            preds.append(pred)
            pos_preds.append(pred_pos)
            
        losses +=_loss
        

    if(train_cond==False):
        y_pred = []
        y_pred_pos = []
        for i in range(num_batches):
            for pred in preds[i]:
                y_pred.append(pred)
            for pred_pos in pos_preds[i]:
                y_pred_pos.append(pred_pos)
        return losses/num_batches, y_pred, y_pred_pos
    
    return losses/num_batches, step

In [16]:
x_id_train, mask_train, y_train, pos_id_train = data_prepare(x_train, label_train, pos_train)
x_id_val, mask_val, y_val, pos_id_val = data_prepare(x_val, label_val, pos_val)

In [17]:
num_epochs = 20
pre_train_period = 1
log_period = 5

for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    pos_id_train = pos_id_train[random]
    
    if(i<pre_train_period):
        losses, step = model(x_id_train, y_train, mask_train, pos_id_train, pre_train=True)
    else:
        losses, step = model(x_id_train, y_train, mask_train, pos_id_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    
    if((i+1)%log_period==0):
        saver.save(sess, save_path=save_dir)                         
        print("Model Saved")
        train_loss, train_pred, train_pred_pos = model(x_id_train, y_train, mask_train, pos_id_train, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_train, train_pred, pos_id_train, train_pred_pos)
        print("Train: F1 : ",  f1_, "Accu: ", accu_, "POS F1 : ",  f1_pos_, "POS Accu: ", accu_pos_, "Loss: ", train_loss)
        
        val_loss, val_pred, val_pred_pos = model(x_id_val, y_val, mask_val, pos_id_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_val, val_pred, pos_id_val, val_pred_pos)
        print("Val: F1 : ",  f1_, "Accu: ", accu_, "POS F1 : ",  f1_pos_, "POS Accu: ", accu_pos_, "Loss: ", val_loss)

Epoch: 1 Step: 43 loss: 19.7660060428
Epoch: 2 Step: 86 loss: 7.90963750107
Epoch: 3 Step: 129 loss: 4.00364196023
Epoch: 4 Step: 172 loss: 3.02462588077
Epoch: 5 Step: 215 loss: 2.8901152389
Model Saved
Train: F1 :  62.2385074704 Accu:  67.6598837209 POS F1 :  93.9333948783 POS Accu:  94.0406976744 Loss:  2.66142181186
Val: F1 :  59.5789388263 Accu:  65.3125 POS F1 :  92.3869189821 POS Accu:  92.5 Loss:  2.96424570084
Epoch: 6 Step: 258 loss: 2.69714353251
Epoch: 7 Step: 301 loss: 2.70068740013
Epoch: 8 Step: 344 loss: 2.60896997951
Epoch: 9 Step: 387 loss: 2.41318572954
Epoch: 10 Step: 430 loss: 2.43129668957
Model Saved
Train: F1 :  71.6399014068 Accu:  75.2906976744 POS F1 :  94.853249207 POS Accu:  94.9127906977 Loss:  2.28305977722
Val: F1 :  66.3664746131 Accu:  71.25 POS F1 :  93.0465015211 POS Accu:  93.125 Loss:  2.78860605955
Epoch: 11 Step: 473 loss: 2.27712107259
Epoch: 12 Step: 516 loss: 2.34832395786
Epoch: 13 Step: 559 loss: 2.18602590505
Epoch: 14 Step: 602 loss: 2.209

In [18]:
num_epochs = 30
pre_train_period = 0
log_period = 5

for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    pos_id_train = pos_id_train[random]
    
    if(i<pre_train_period):
        losses, step = model(x_id_train, y_train, mask_train, pos_id_train, pre_train=True)
    else:
        losses, step = model(x_id_train, y_train, mask_train, pos_id_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    
    if((i+1)%log_period==0):
        saver.save(sess, save_path=save_dir)                         
        print("Model Saved")
        train_loss, train_pred, train_pred_pos = model(x_id_train, y_train, mask_train, pos_id_train, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_train, train_pred, pos_id_train, train_pred_pos)
        print("Train: F1 : ",  f1_, "Accu: ", accu_, "POS F1 : ",  f1_pos_, "POS Accu: ", accu_pos_, "Loss: ", train_loss)
        
        val_loss, val_pred, val_pred_pos = model(x_id_val, y_val, mask_val, pos_id_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_val, val_pred, pos_id_val, val_pred_pos)
        print("Val: F1 : ",  f1_, "Accu: ", accu_, "POS F1 : ",  f1_pos_, "POS Accu: ", accu_pos_, "Loss: ", val_loss)

Epoch: 1 Step: 903 loss: 1.60123486852
Epoch: 2 Step: 946 loss: 1.66395208725
Epoch: 3 Step: 989 loss: 1.70490837652
Epoch: 4 Step: 1032 loss: 1.55862255568
Epoch: 5 Step: 1075 loss: 1.70186423701
Model Saved
Train: F1 :  80.7070151164 Accu:  82.7398255814 POS F1 :  97.0772764082 POS Accu:  97.0930232558 Loss:  1.73972274259
Val: F1 :  74.868120971 Accu:  77.03125 POS F1 :  94.8264585492 POS Accu:  94.84375 Loss:  2.81427048445
Epoch: 6 Step: 1118 loss: 1.55971108342
Epoch: 7 Step: 1161 loss: 1.5758984976
Epoch: 8 Step: 1204 loss: 1.50112192021
Epoch: 9 Step: 1247 loss: 1.56894098049
Epoch: 10 Step: 1290 loss: 1.37336764918
Model Saved
Train: F1 :  77.1270195676 Accu:  81.3590116279 POS F1 :  98.4986606875 POS Accu:  98.5101744186 Loss:  1.4341281001
Val: F1 :  74.3986316498 Accu:  77.96875 POS F1 :  97.1642099666 POS Accu:  97.1875 Loss:  2.1966968894
Epoch: 11 Step: 1333 loss: 1.54621222269
Epoch: 12 Step: 1376 loss: 1.63824103184
Epoch: 13 Step: 1419 loss: 1.50091427703
Epoch: 14 St

In [19]:
num_epochs = 20
pre_train_period = 0
log_period = 5

for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    pos_id_train = pos_id_train[random]
    
    if(i<pre_train_period):
        losses, step = model(x_id_train, y_train, mask_train, pos_id_train, pre_train=True)
    else:
        losses, step = model(x_id_train, y_train, mask_train, pos_id_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    
    if((i+1)%log_period==0):
        saver.save(sess, save_path=save_dir)                         
        print("Model Saved")
        train_loss, train_pred, train_pred_pos = model(x_id_train, y_train, mask_train, pos_id_train, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_train, train_pred, pos_id_train, train_pred_pos)
        print("Train: F1 : ",  f1_, "Accu: ", accu_, "POS F1 : ",  f1_pos_, "POS Accu: ", accu_pos_, "Loss: ", train_loss)
        
        val_loss, val_pred, val_pred_pos = model(x_id_val, y_val, mask_val, pos_id_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_val, val_pred, pos_id_val, val_pred_pos)
        print("Val: F1 : ",  f1_, "Accu: ", accu_, "POS F1 : ",  f1_pos_, "POS Accu: ", accu_pos_, "Loss: ", val_loss)

Epoch: 1 Step: 2193 loss: 1.3766035895
Epoch: 2 Step: 2236 loss: 1.29416433046
Epoch: 3 Step: 2279 loss: 1.31451716534
Epoch: 4 Step: 2322 loss: 1.34232876883
Epoch: 5 Step: 2365 loss: 1.28058474286
Model Saved
Train: F1 :  81.6897913296 Accu:  84.4840116279 POS F1 :  98.6051563971 POS Accu:  98.6191860465 Loss:  1.32602877118
Val: F1 :  78.105125476 Accu:  81.40625 POS F1 :  97.7917763385 POS Accu:  97.8125 Loss:  1.9950668335
Epoch: 6 Step: 2408 loss: 1.49707775199
Epoch: 7 Step: 2451 loss: 1.46867943365
Epoch: 8 Step: 2494 loss: 1.22669876869
Epoch: 9 Step: 2537 loss: 1.2759284183
Epoch: 10 Step: 2580 loss: 1.31809062459
Model Saved
Train: F1 :  82.8268451801 Accu:  84.7020348837 POS F1 :  98.6454774199 POS Accu:  98.6555232558 Loss:  1.4226906161
Val: F1 :  77.2141144635 Accu:  79.53125 POS F1 :  96.8573224387 POS Accu:  96.875 Loss:  2.13040351868
Epoch: 11 Step: 2623 loss: 1.24871666487
Epoch: 12 Step: 2666 loss: 1.30132872559
Epoch: 13 Step: 2709 loss: 1.30720291443
Epoch: 14 St

In [ ]:
saver.restore(sess, save_dir)